# Storing Dataset  

This notebook focusses on connecting to the Public GCP dataset and then from there used to be able to save the snippet of data as a parquet file

In [ ]:
#Install necessary libraries
%pip install --upgrade google-cloud-bigquery pyarrow
%pip install pandas numpy matplotlib seaborn
%pip install db-dtypes
%pip install time datetime
%pip install scikit-learn
%pip install google-cloud-bigquery-storage

In [2]:
import os
import pandas as pd
from google.cloud import bigquery

# Set your GCP project and BigQuery location
project = "london-bike-hire-dataset-test"
location = "EU"

# Set path to your credentials JSON file (local path)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\msidh\Documents\Coding\London Bicycle Hires App\credentials\bq_viewer_key.json"

# Initialise BigQuery client
client = bigquery.Client(project=project, location=location)

#List all tables in the london_bicycles dataset
#should return cycle_hire and cycle_stations

dataset_ref = client.dataset("london_bicycles", project="bigquery-public-data")
tables = client.list_tables(dataset_ref)

print("Tables in bigquery-public-data.london_bicycles:")
for table in tables:
    print(" -", table.table_id)


Tables in bigquery-public-data.london_bicycles:
 - cycle_hire
 - cycle_stations


In [4]:
cycle_df = client.query("SELECT * FROM `bigquery-public-data.london_bicycles.cycle_hire` LIMIT 1").to_dataframe()

# To display the column names:
print(cycle_df.columns.tolist())

['rental_id', 'duration', 'duration_ms', 'bike_id', 'bike_model', 'end_date', 'end_station_id', 'end_station_name', 'start_date', 'start_station_id', 'start_station_name', 'end_station_logical_terminal', 'start_station_logical_terminal', 'end_station_priority_id']


In [ ]:
from pathlib import Path
from google.cloud import bigquery, bigquery_storage_v1
import pandas as pd

# Define the bronze storage path
bronze_store = Path("storage/Bronze")
bronze_store.mkdir(parents=True, exist_ok=True)

# Instantiate BigQuery clients
client = bigquery.Client()
bqstorage_client = bigquery_storage_v1.BigQueryReadClient()

# ---- A. Fetch 2022 trips -----------------------------------------------------
sql_trips_22 = """
SELECT *
FROM  `bigquery-public-data.london_bicycles.cycle_hire`
WHERE EXTRACT(YEAR FROM start_date) = 2022
"""
df_trips_22 = client.query(sql_trips_22).to_dataframe(bqstorage_client=bqstorage_client)
df_trips_22.to_parquet(bronze_store / "cycle_hire_2022.parquet", compression="snappy", index=False)
print("Trips saved to storage/Bronze/cycle_hire_2022.parquet")

# ---- B. Fetch station reference ---------------------------------------------
sql_stations = "SELECT * FROM `bigquery-public-data.london_bicycles.cycle_stations`"
df_stations = client.query(sql_stations).to_dataframe(bqstorage_client=bqstorage_client)
df_stations.to_parquet(bronze_store / "cycle_stations.parquet", compression="snappy", index=False)
print("Stations saved to storage/Bronze/cycle_stations.parquet")


✅ Trips saved to storage/Bronze/cycle_hire_2022.parquet
✅ Stations saved to storage/Bronze/cycle_stations.parquet
